Imports

In [ ]:
from datetime import datetime
import time

from contracts_lib_py.account import Account
from common_utils_py.agreements.service_types import ServiceTypesIndices


from nevermined_sdk_py import Config, Nevermined
from nevermined_sdk_py.nevermined.keeper import NeverminedKeeper as Keeper

CONSUMER_ADDRESS = "0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e"
CONSUMER_PASSWORD = "node0"
CONSUMER_KEYFILE = "../resources/accounts/consumer.json"
CONFIG_FILE = "../config.ini"

import pathlib
print(pathlib.Path(CONFIG_FILE).resolve().as_posix())

PROVIDER_ADDRESS = "0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0"
ASSET_COMPUTE_DID_1 = "did:nv:98721b32acde006a97f4a4b2ac4f1d508cae74cdd4351bff651c471c11296275"
ASSET_COMPUTE_DID_2 = "did:nv:c691207e76202bba58710f9e27f55f7c06b51e9ebaa4c5d80b06636d23aca213"
ASSET_COORDINATOR_DID = "did:nv:5f5f7523e1e5ed2c3780e9be8ae7b05ca0953ee09ce7f19bb13ef36607b3e978"

Setup Nevermined and accounts

In [ ]:
nevermined = Nevermined(Config(CONFIG_FILE))
consumer_account = Account(CONSUMER_ADDRESS, CONSUMER_PASSWORD, CONSUMER_KEYFILE)

nevermined.accounts.request_tokens(consumer_account, 10)


Publish algorithm

In [ ]:
metadata_algorithm = {
    "main": {
        "name": "Fraud Demo Algorithm",
        "dateCreated": datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "author": "Nevermined Consumer",
        "license": "",
        "price": "0",
        "files": [
            {
                "index": 0,
                "contentType": "text/text",
                "checksum": "0x52b5c93b82dd9e7ecc3d9fdf4755f7f69a54484941897dc517b4adfe3bbc3377",
                "checksumType": "MD5",
                "contentLength": "12057507",
                "url": "https://raw.githubusercontent.com/nevermined-io/fl-demo/feature/xain-fraud-detection/fraud-detection-xain/notebooks/federated_fraud_demo.ipynb"
            }
        ],
        "type": "algorithm",
        "algorithm": {
            "language": "python",
            "format": "py",
            "version": "0.1.0",
            "entrypoint": (
                "pip install jupyter shfl pandas papermill xain-sdk && "
                "papermill --stdout-file - federated_fraud_demo.ipynb $NEVERMINED_OUTPUTS_PATH/federated_fraud_demo_output.ipynb"
            ),
            "requirements": {
                "container": {
                    "image": "python",
                    "tag": "3.8-slim-buster",
                    "checksum": "sha256:53ad3a03b2fb240b6c494339821e6638cd44c989bcf26ec4d51a6a52f7518c1d"
                }
            }           
        }
    }
}

ddo_algorithm = nevermined.assets.create(metadata_algorithm, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Algorithm DID: {ddo_algorithm.did}")

Publish the workflows:
- Two for the compute to the data assets
- One for the coordinator service

In [ ]:
# Compute asset 1
metadata_workflow1 = {
    "main": {
        "name": "Fraud Demo Workflow 1",
        "dateCreated": datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "author": "Nevermined Consumer",
        "license": "",
        "price": "0",
        "type": "workflow",
        "workflow": {
            "stages": [
                {
                    "index": 0,
                    "input": [
                        {
                            "index": 0,
                            "id": ASSET_COMPUTE_DID_1
                        }
                    ],
                    "transformation": {
                        "id": ddo_algorithm.did
                    }
                }
            ]
        }
    }
}

ddo_workflow1 = nevermined.assets.create(metadata_workflow1, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow1.did}")

# Compute asset 2
metadata_workflow2 = {
    "main": {
        "name": "Fraud Demo Workflow 2",
        "dateCreated": datetime.utcnow().isoformat(timespec="seconds") + "Z",
        "author": "Nevermined Consumer",
        "license": "",
        "price": "0",
        "type": "workflow",
        "workflow": {
            "stages": [
                {
                    "index": 0,
                    "input": [
                        {
                            "index": 0,
                            "id": ASSET_COMPUTE_DID_2
                        }
                    ],
                    "transformation": {
                        "id": ddo_algorithm.did
                    }
                }
            ]
        }
    }
}

ddo_workflow2 = nevermined.assets.create(metadata_workflow2, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow2.did}")

# Coordinator service
metadata_workflow_coordinator = {
  "main": {
    "author": "Nevermined Consumer",
    "checksum": "0x52b5c93b82dd9e7ecc3d9fdf4755f7f69a54484941897dc517b4adfe3bbc3377",
    "dateCreated": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "datePublished": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "license": "",
    "name": "My workflow",
    "price": "0",
    "type": "fl-coordinator",
    "parameters": {
      "minParticipants": 2,
      "participantsRatio": 1,
      "rounds": 10
    },
    "workflow": {
      "stages": [
        {
          "input": [],
          "index": 0,
          "requirements": {
            "serverInstances": 1,
            "container": {
              "image": "rmarques/xain-fl",
              "tag": "latest",
              "checksum": "sha256:cb57ecfa6ebbefd8ffc7f75c0f00e57a7fa739578a429b6f72a0df19315deadc"
            }
          }
        }
      ]
    }
  }
}

ddo_workflow_coordinator = nevermined.assets.create(metadata_workflow_coordinator, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow_coordinator.did}")

Order computations:
- One for each data asset
- One for the coordinator service

In between we want for the blockchain to process the request.

In [ ]:
keeper = Keeper.get_instance()

service_agreement_id1 = nevermined.assets.order(ASSET_COMPUTE_DID_1, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account)
print(f"Service Agreement ID: {service_agreement_id1}")

event = keeper.lock_reward_condition.subscribe_condition_fulfilled(
        service_agreement_id1, 60, None, (), wait=True
    )
assert event is not None, "Reward condition is not found"

service_agreement_id2 = nevermined.assets.order(ASSET_COMPUTE_DID_2, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account)
print(f"Service Agreement ID: {service_agreement_id2}")

event = keeper.lock_reward_condition.subscribe_condition_fulfilled(
        service_agreement_id2, 60, None, (), wait=True
    )
assert event is not None, "Reward condition is not found"

service_agreement_id_coordinator = nevermined.assets.order(ASSET_COORDINATOR_DID, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account)
print(f"Service Agreement ID: {service_agreement_id_coordinator}")

event = keeper.lock_reward_condition.subscribe_condition_fulfilled(
        service_agreement_id_coordinator, 60, None, (), wait=True
    )
assert event is not None, "Reward condition is not found"


Execute workflows

This orders the Nevermined to start the execution of the workflows:
- One for each data asset
- One for the coordinator service

In [ ]:
keeper = Keeper.get_instance()

print(keeper.artifacts_path)

In [ ]:
execution_id1 = nevermined.assets.execute(
    service_agreement_id1,
    ASSET_COMPUTE_DID_1,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow1.did
)
print(f"Execution ID: {execution_id1}")

execution_id2 = nevermined.assets.execute(
    service_agreement_id2,
    ASSET_COMPUTE_DID_2,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow2.did
)
print(f"Execution ID: {execution_id2}")

execution_id_coordinator = nevermined.assets.execute(
    service_agreement_id_coordinator,
    ASSET_COORDINATOR_DID,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow_coordinator.did
)
print(f"Execution ID: {execution_id_coordinator}")

Wait for compute job to finish

In [ ]:
status = None
while status not in ["Succeeded", "Failed"]:
    result = nevermined.assets.compute_status(service_agreement_id_coordinator, execution_id_coordinator, consumer_account)
    status = result["status"]
    print(f"{execution_id_coordinator}: {status}")
    time.sleep(10)

output_did = result["did"]
print(f"Output DID: {output_did}")

In [ ]:
nevermined.assets.download(output_did, ServiceTypesIndices.DEFAULT_ACCESS_INDEX, consumer_account, "./")